In [1]:
import config
import pandas as pd
import psycopg2
import numpy as np

- Code acte consultation : « CNP »
- Base tarif : 42,5 (Secteur 2 non OPTAM)
- MPC : majoration du médecin spécialiste : 2,7 euros
- Donc S1 et S2 OPTAM = 45,2
- keep if strpos(v17,"CNP")
- MCS: MCS : majoration de coordination pour les psychiatres : 5 euros

In [2]:
study_id = 3
sql = f"select * from depassement_study where id={study_id}"
df_study = pd.read_sql(sql, config.connection_string)
df_study

,id,profession_type,datesource_min,datesource_max,tarif_s1
0,3,Cardiologue,2301,2308,51.0


In [3]:
profession_type = df_study.iloc[0]["profession_type"]
datesource_min = df_study.iloc[0]["datesource_min"]
datesource_max = df_study.iloc[0]["datesource_max"]
tarif_s1 = df_study.iloc[0]["tarif_s1"]
profession_type, datesource_min, datesource_max, tarif_s1

('Cardiologue', 2301, 2308, 51.0)

In [4]:
sql = f"""
select p.*, t.*, tds.date_source_id, b.id as adresse_id, an.cp as cp, ar.dept_id as dept_id, b.code_insee from ps p
join tarif t on t.ps_id = p.id
join tarif_date_source tds on tds.tarif_id = t.id
join cabinet c on t.cabinet_id = c.id
join adresse_raw ar on c.adresse_raw_id = ar.id
join adresse_norm an on ar.adresse_norm_id = an.id
join ban b on an.ban_id = b.id
--join ps_cabinet_date_source pcd on pcd.ps_id = p.id and pcd.cabinet_id = c.id and pcd.date_source_id = tds.date_source_id
join profession_type pt on pt.profession = '{profession_type}' and t.profession_id = pt.profession_id
where tds.date_source_id >= {datesource_min} and  tds.date_source_id <= {datesource_max}
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

df = pd.read_csv("../data/depassement/cardiologues.csv", low_memory=False)
# df = pd.read_sql(sql, config.connection_string)  #63472
df  #309601





,id,genre,key,nom,prenom,has_inpp,rule_nb,id.1,profession_id,mode_exercice_id,nature_id,convention_id,option_contrat,vitale,code,ps_id,cabinet_id,famille_acte_id,montant,borne_inf,borne_sup,montant_2,borne_inf_2,borne_sup_2,montant_imagerie,borne_inf_imagerie,borne_sup_imagerie,montant_anesthesie,borne_inf_anesthesie,borne_sup_anesthesie,montant_cec,borne_inf_cec,borne_sup_cec,tarif_id,date_source_id,id.2,nom.1,key.1,telephone,adresse_raw_id,id.3,adresse2,adresse3,adresse4,cp,commune,adresse_norm_id,dept_id,id.4,numero,rue1,rue2,cp.1,commune.1,osm_id,osm_score,ban_id,ban_score,source_id,lon,lat,score,dept_id.1,iris,id.5,adresse_id,numero.1,rep,nom_voie,code_postal,code_insee,nom_commune,nom_ancienne_commune,lon.1,lat.1,libelle_acheminement,nom_afnor,dept_id.2,is_lieu_dit
0,650220,H,810003462099,IMIANITOFF,MANUEL,True,1.0,19957581,6,NaN,4,3,False,True,CSC,650220,2554,NaN,80.0,47.7,82.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19957581,2301,2554,IMIANITOFF MANUEL,IMIANITOFF_MANUEL_06600_4_AVENUE_ROBERT_SOLEAU,493340041.0,2106,2106,RESIDENCE LE COLISEE,4 AVENUE ROBERT SOLEAU,NaN,6600,ANTIBES,2049,6,2049,4.0,AVENUE ROBERT SOLEAU,RESIDENCE LE COLISEE,6600,ANTIBES,NaN,NaN,825565,1.000000,2,7.12090,43.581045,1.000000,6,060040114,825565,06004_2970_00004,4.0,NaN,AVENUE ROBERT SOLEAU,6600,6004,ANTIBES,NaN,7.12090,43.581045,NaN,NaN,6,False
1,650220,H,810003462099,IMIANITOFF,MANUEL,True,1.0,19957581,6,NaN,4,3,False,True,CSC,650220,2554,NaN,80.0,47.7,82.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19957581,2302,2554,IMIANITOFF MANUEL,IMIANITOFF_MANUEL_06600_4_AVENUE_ROBERT_SOLEAU,493340041.0,2106,2106,RESIDENCE LE COLISEE,4 AVENUE ROBERT SOLEAU,NaN,6600,ANTIBES,2049,6,2049,4.0,AVENUE ROBERT SOLEAU,RESIDENCE LE COLISEE,6600,ANTIBES,NaN,NaN,825565,1.000000,2,7.12090,43.581045,1.000000,6,060040114,825565,06004_2970_00004,4.0,NaN,AVENUE ROBERT SOLEAU,6600,6004,ANTIBES,NaN,7.12090,43.581045,NaN,NaN,6,False
2,650220,H,810003462099,IMIANITOFF,MANUEL,True,1.0,19957581,6,NaN,4,3,False,True,CSC,650220,2554,NaN,80.0,47.7,82.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19957581,2303,2554,IMIANITOFF MANUEL,IMIANITOFF_MANUEL_06600_4_AVENUE_ROBERT_SOLEAU,493340041.0,2106,2106,RESIDENCE LE COLISEE,4 AVENUE ROBERT SOLEAU,NaN,6600,ANTIBES,2049,6,2049,4.0,AVENUE ROBERT SOLEAU,RESIDENCE LE COLISEE,6600,ANTIBES,NaN,NaN,825565,1.000000,2,7.12090,43.581045,1.000000,6,060040114,825565,06004_2970_00004,4.0,NaN,AVENUE ROBERT SOLEAU,6600,6004,ANTIBES,NaN,7.12090,43.581045,NaN,NaN,6,False
3,650220,H,810003462099,IMIANITOFF,MANUEL,True,1.0,19957581,6,NaN,4,3,False,True,CSC,650220,2554,NaN,80.0,47.7,82.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19957581,2304,2554,IMIANITOFF MANUEL,IMIANITOFF_MANUEL_06600_4_AVENUE_ROBERT_SOLEAU,493340041.0,2106,2106,RESIDENCE LE COLISEE,4 AVENUE ROBERT SOLEAU,NaN,6600,ANTIBES,2049,6,2049,4.0,AVENUE ROBERT SOLEAU,RESIDENCE LE COLISEE,6600,ANTIBES,NaN,NaN,825565,1.000000,2,7.12090,43.581045,1.000000,6,060040114,825565,06004_2970_00004,4.0,NaN,AVENUE ROBERT SOLEAU,6600,6004,ANTIBES,NaN,7.12090,43.581045,NaN,NaN,6,False
4,650220,H,810003462099,IMIANITOFF,MANUEL,True,1.0,19957581,6,NaN,4,3,False,True,CSC,650220,2554,NaN,80.0,47.7,82.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19957581,2305,2554,IMIANITOFF MANUEL,IMIANITOFF_MANUEL_06600_4_AVENUE_ROBERT_SOLEAU,493340041.0,2106,2106,RESIDENCE LE COLISEE,4 AVENUE ROBERT SOLEAU,NaN,6600,ANTIBES,2049,6,2049,4.0,AVENUE ROBERT SOLEAU,RESIDENCE LE COLISEE,6600,ANTIBES,NaN,NaN,825565,1.000000,2,7.12090,43.581045,1.000000,6,060040114,825565,06004_2970_00004,4.0,NaN,AVENUE ROBERT SOLEAU,6600,6004,ANTIBES,NaN,7.12090,43.581045,NaN,NaN,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309596,649769,H,81

In [5]:
# Renommer les colonnes
df.rename(columns={
    'genre': 'gender',
    'prenom': 'prénom',
    'code': 'codeccamdelacte',
    'nature_id': 'naturedexercice',
    'convention_id': 'convention',
    'code_insee': 'codeinsee',
    'cp': 'matchcp',
    'montant': 'montantgénéralementconstaté',
    'borne_inf': 'borneinférieuredumontant',
    'borne_sup': 'bornesupérieuredumontant',
    'option_contrat': 'optioncontratdaccèsauxsoins',
    'dept_id': 'dep',
}, inplace=True)

# Colonnes à conserver
df = df[['ps_id', 'gender', 'nom', 'prénom', 'codeccamdelacte', 'naturedexercice', 'convention',
         'codeinsee', 'montantgénéralementconstaté', 'borneinférieuredumontant', 'bornesupérieuredumontant',
         'adresse_id', 'date_source_id', 'optioncontratdaccèsauxsoins', 'matchcp', 'dep']]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep
0,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2301,False,6600,6
1,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2302,False,6600,6
2,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2303,False,6600,6
3,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2304,False,6600,6
4,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2305,False,6600,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309596,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2304,True,6800,6
309597,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2305,True,6800,6
309598,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2306,True,6800,6
309599,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2307,True,6800,6


In [6]:
# Clé combinée
pd.options.mode.copy_on_write = True
df['b'] = df['ps_id'].astype(str) + "_" + df['adresse_id'].astype(str)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
0,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2301,False,6600,6,650220_06004_2970_00004
1,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2302,False,6600,6,650220_06004_2970_00004
2,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2303,False,6600,6,650220_06004_2970_00004
3,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2304,False,6600,6,650220_06004_2970_00004
4,650220,H,IMIANITOFF,MANUEL,CSC,4,3,6004,80.0,47.7,82.7,06004_2970_00004,2305,False,6600,6,650220_06004_2970_00004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309596,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2304,True,6800,6,649769_06027_2055_00061
309597,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2305,True,6800,6,649769_06027_2055_00061
309598,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2306,True,6800,6,649769_06027_2055_00061
309599,649769,H,PELLETIER,ROBERT,EDQM0010,2,3,6027,0.0,0.0,0.0,06027_2055_00061,2307,True,6800,6,649769_06027_2055_00061


In [7]:
nb_ps_id = df["ps_id"].nunique()
nb_ps_id  # 4533


4533

In [8]:
nb_b = df["b"].nunique()
nb_b  # 5718

5718

In [9]:
df = df.sort_values(by='ps_id')
df['unique'] = df.groupby('ps_id')['adresse_id'].transform('nunique')
df['un'] = df['unique']
pd.set_option('display.max_rows', 200)
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un
289610,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1
289614,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2303,False,1630,1,647961_01354_0245_00110,1,1
289615,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2304,False,1630,1,647961_01354_0245_00110,1,1
289616,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1
289617,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2306,False,1630,1,647961_01354_0245_00110,1,1
289618,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1
289619,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2308,False,1630,1,647961_01354_0245_00110,1,1
289595,647961,H,FAVIA,AGOSTINO,DEQP0030,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2308,False,1630,1,647961_01354_0245_00110,1,1
289594,647961,H,FAVIA,AGOSTINO,DEQP0030,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1
289592,647961,H,FAVIA,AGOSTINO,DEQP0030,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1


In [10]:
# Comptages globaux
df['NB_total'] = df.groupby('dep')['ps_id'].transform('nunique')
df = df.sort_values(by='dep')
df["NB_total"].unique()

array([ 23,  29,  21,  13,  10, 140,  16,  11,   3,  25, 257,  59,   6,
        17,  47,  39,  26,   2,  18,  27,  19,  28,  43,  53, 135,   8,
       155, 109,  63,   7,  58,  83,   9,  30,  14,  46, 103,  38,  40,
        66,  85, 171,  36,  12,  84,  54,  78,  15,  42,  98,  65, 158,
         4,  24, 297,  73, 113,  41,  97,  49, 118, 164, 126,  95,  57],
      dtype=int64)

In [11]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2_total'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2_total'] = df.groupby('dep')['NB2_total'].transform('max')
df["NB2_total"] = df["NB2_total"].fillna(0)
df["NB2_total"].unique()



array([ 10.,   2.,   4.,  72.,   5.,   0.,   1.,  65.,  18.,  11.,  12.,
         6.,   3.,   7.,  22.,  87.,  17.,  28.,  61.,  14.,  23.,  45.,
        21., 119.,  27., 200.,  35.,  66.,   9.,  56., 115.,  53.,  48.,
        15.,   8.])

In [12]:
df['c'] = 1
df['NB_Ftotal'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_Ftotal"].unique() # 4533

array([4533], dtype=int64)

In [13]:
df['NB2_Ftotal'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_Ftotal'] = df['NB2_Ftotal'].max()  # Car ne diot pas avoir de nan
df["NB2_Ftotal"].unique() # 1517

array([1517.])

In [14]:
# Pondération
df['weight'] = df['un'].map({1: 1, 2: 0.5, 3: 0.33, 4: 0.25, 5: 0.2}).fillna(0)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
289610,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
64244,757551,H,HALARY,JEAN JACQUES,EBQM0010,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2308,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
64245,757551,H,HALARY,JEAN JACQUES,DKRP0040,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2301,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
64246,757551,H,HALARY,JEAN JACQUES,DKRP0040,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2302,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
157741,742917,H,COYAULT,JEAN MARC,YYYY1720,3,3,1283,66.0,0.0,0.0,01283_0940_00019,2306,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107620,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2305,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107621,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2306,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107622,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2307,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107624,659511,H,MASSIANI,PIERRE FRANCOIS,APC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2301,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0


In [15]:
# Tous les praticiens ne font pas de consultation
nb_ps_id, df["ps_id"].nunique()

(4533, 4533)

In [16]:
df = df[df['codeccamdelacte'].str.contains("CS")]
df #50737

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
289610,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
157756,742917,H,COYAULT,JEAN MARC,CS_,3,3,1283,23.0,23.0,60.0,01283_0940_00019,2305,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.0
64324,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2308,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
64323,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2307,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
64322,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2306,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107618,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2303,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107619,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2304,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107620,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2305,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0
107621,659511,H,MASSIANI,PIERRE FRANCOIS,CSC+MCC,3,1,2B033,0.0,0.0,0.0,2b033_0418_00008,2306,False,20200,202,659511_2b033_0418_00008,1,1,12,1.0,1,4533,1517.0,1.0


In [17]:
df["codeccamdelacte"].value_counts(normalize=True)

codeccamdelacte
CSC+MCC            0.417565
CS_+MPC+MCS        0.347616
CSC                0.135936
CS_+MPC            0.064706
CS_                0.033881
CS_+MPC+MCS+MCX    0.000158
CS_+MPC+MCS+MTX    0.000138
Name: proportion, dtype: float64

In [18]:
df = df.sort_values(by='convention')
df.groupby('convention')["ps_id"].nunique() #2755 5 938

convention
1    2755
2       5
3     938
Name: ps_id, dtype: int64

In [19]:
df = df.sort_values(by=['b',"convention", "codeccamdelacte", "date_source_id"])
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289613,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289614,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2303,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289615,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2304,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289616,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289617,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2306,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289618,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289619,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2308,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289604,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0
289605,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0


In [20]:
# Moyenne par groupe
df['mp'] = df.groupby(['b', 'convention', 'codeccamdelacte'])['montantgénéralementconstaté'].transform('mean')
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289613,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289614,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2303,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289615,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2304,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289616,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289617,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2306,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289618,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2307,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289619,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2308,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289604,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289605,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0


In [21]:
# 28083
df = df[(df['codeccamdelacte'] == "CSC") | (df['codeccamdelacte'] == "CSC+MCC")]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289613,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289614,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2303,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289615,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2304,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
289616,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268117,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2307,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.0,0.0
268118,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2308,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.0,0.0
228183,799718,F,PEDELHEZ,KARINE,CSC+MCC,4,1,59606,0.0,0.0,0.0,59606_1140_00114,2306,False,59322,59,799718_59606_1140_00114,1,1,171,61.0,1,4533,1517.0,1.0,0.0
228171,799718,F,PEDELHEZ,KARINE,CSC+MCC,5,1,59606,0.0,0.0,0.0,59606_1140_00114,2307,False,59322,59,799718_59606_1140_00114,1,1,171,61.0,1,4533,1517.0,1.0,0.0


In [22]:
tarif_s1

51.0

In [23]:
print(len(df[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False)]))
df.loc[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False), 'mp'] = tarif_s1
df


19507


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0
289613,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2302,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0
289614,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2303,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0
289615,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2304,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0
289616,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2305,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268117,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2307,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.0,51.0
268118,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2308,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.0,51.0
228183,799718,F,PEDELHEZ,KARINE,CSC+MCC,4,1,59606,0.0,0.0,0.0,59606_1140_00114,2306,False,59322,59,799718_59606_1140_00114,1,1,171,61.0,1,4533,1517.0,1.0,51.0
228171,799718,F,PEDELHEZ,KARINE,CSC+MCC,5,1,59606,0.0,0.0,0.0,59606_1140_00114,2307,False,59322,59,799718_59606_1140_00114,1,1,171,61.0,1,4533,1517.0,1.0,51.0


****on élimine les duplications ps-adresse convention acte puis on sélectionne le prix le plus élevé d'un acte de consultation (discard tarif opposable des secteurs 2)
***puis on élimine les duplications ps-adresse convention (car les actes ont le même prix maintenant pour chaque ps-adresse)

In [24]:
# 3642

df = df.drop_duplicates(subset=['b', 'convention', 'codeccamdelacte'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0
289604,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70102,799626,F,CHANAUD,SANDRINE,CSC+MCC,3,1,67180,0.0,0.0,0.0,67180_0911_00005,2302,False,67500,67,799626_67180_0911_00005,1,1,98,45.0,1,4533,1517.0,1.00,51.0
18957,799677,H,CERQUEIRA,SEBASTIEN,CSC+MCC,2,1,76617,0.0,0.0,0.0,76617_0081_00181,2304,False,76160,76,799677_76617_0081_00181,1,1,73,22.0,1,4533,1517.0,1.00,51.0
297960,799692,F,HAFFNER,CLAIRE,CSC+MCC,3,3,38185,51.0,51.0,51.0,38185_2150_00008,2305,True,38000,38,799692_38185_2150_00008,1,1,83,28.0,1,4533,1517.0,1.00,51.0
268116,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2306,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.00,51.0


In [25]:
df = df.sort_values(by=['b',"convention"])
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0
289604,647961,H,FAVIA,AGOSTINO,CSC+MCC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7
289372,648130,H,BOBILLIER,MARC,CSC,3,1,71417,0.0,0.0,0.0,71417_0580_00004,2301,False,71460,71,648130_71417_0580_00004,1,1,26,3.0,1,4533,1517.0,1.00,51.0
190194,648160,H,MAYAUD,NORBERT,CSC,3,3,1053,47.7,47.7,70.0,01053_1950_00060,2301,True,1000,1,648160_01053_1950_00060,1,1,23,10.0,1,4533,1517.0,1.00,47.7
266357,648279,H,BOGE,ANTOINE,CSC+MCC,3,1,84007,0.0,0.0,0.0,84007_3200_01750,2301,False,84082,84,648279_84007_3200_01750,1,1,49,11.0,1,4533,1517.0,1.00,51.0
190252,648335,H,TANAZACQ,REMI,CSC,3,1,1053,80.0,47.7,80.7,01053_1950_00060,2301,True,1000,1,648335_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,80.0
140196,648477,H,HENON,PIERRE,CSC+MCC,5,1,2691,0.0,0.0,0.0,02691_2w5ppd_00001,2301,False,2100,2,648477_02691_2w5ppd_00001,1,1,29,2.0,1,4533,1517.0,1.00,51.0


In [26]:
df['prixmoyen'] = df.groupby(['b', 'convention'])['mp'].transform('max')
df.head(20)
df['prixmoyen'].unique()

array([ 51. , 100.7,  47.7,  80. ,  68. ,  82. , 150. ,  65. ,  90. ,
        70. ,  81. , 107.7,  73.4, 122.7,  72.7, 102.7,  71. ,   0. ,
       100. ,  60. ,  47.7,  55. ,  93. ,  66.7,  72. ,  69. ,  61. ,
        74. ,  50. ,  67. ,  75. ,  81.7,  92.7,  76. ,  70.7,  74.2,
       110. ,  62. ,  65.2,  89. ,  77. ,  87.7,  98. ,  84.9,  85. ,
       139.7, 120. , 125.7, 130. , 140. , 135. , 108. , 124. ,  95. ,
        78. ,  86. ,  67.7, 102. ,  92. ,  78.3,  49. ,  80.7,  90.7,
        77.7,  82.7,  62.7, 105. ,  63. ,  66. ,  57. , 152. ,  79. ,
        75.4,  73. ,  97.7,  58. ,  71.5, 170. ,  84.5,  54.6,  57.7,
        70.4,  64. ,  72.9,  56.5,  97.3,  65.7,  65.4])

In [27]:
# Reste 3303
df = df.drop_duplicates(subset=['b', 'convention'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7,100.7
289372,648130,H,BOBILLIER,MARC,CSC,3,1,71417,0.0,0.0,0.0,71417_0580_00004,2301,False,71460,71,648130_71417_0580_00004,1,1,26,3.0,1,4533,1517.0,1.00,51.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70102,799626,F,CHANAUD,SANDRINE,CSC+MCC,3,1,67180,0.0,0.0,0.0,67180_0911_00005,2302,False,67500,67,799626_67180_0911_00005,1,1,98,45.0,1,4533,1517.0,1.00,51.0,51.0
18957,799677,H,CERQUEIRA,SEBASTIEN,CSC+MCC,2,1,76617,0.0,0.0,0.0,76617_0081_00181,2304,False,76160,76,799677_76617_0081_00181,1,1,73,22.0,1,4533,1517.0,1.00,51.0,51.0
297960,799692,F,HAFFNER,CLAIRE,CSC+MCC,3,3,38185,51.0,51.0,51.0,38185_2150_00008,2305,True,38000,38,799692_38185_2150_00008,1,1,83,28.0,1,4533,1517.0,1.00,51.0,51.0
268116,799707,F,PUJADAS,PENELOPE,CSC+MCC,3,1,30189,0.0,0.0,0.0,30189_4340_00009,2306,False,30000,30,799707_30189_4340_00009,1,1,53,7.0,1,4533,1517.0,1.00,51.0,51.0


In [28]:
df.groupby('convention')["ps_id"].nunique()

convention
1    2390
2       4
3     825
Name: ps_id, dtype: int64

In [29]:
# df = df.drop_duplicates(subset=['b']) # inutile
print(len(df[df["prixmoyen"] == 0])) #2
print(len(df[df["prixmoyen"] < tarif_s1])) #78 /!\
df['prixmoyen'] = df['prixmoyen'].replace(0, tarif_s1)
df.loc[df['prixmoyen'] < tarif_s1, 'prixmoyen'] = tarif_s1
# ////Prix OK ////
df['prixmoyen'].unique()

4
82


array([ 51. , 100.7,  80. ,  68. ,  82. , 150. ,  65. ,  90. ,  70. ,
        81. , 107.7,  73.4, 122.7,  72.7, 102.7,  71. , 100. ,  60. ,
        55. ,  93. ,  66.7,  72. ,  69. ,  61. ,  74. ,  67. ,  75. ,
        81.7,  92.7,  76. ,  70.7,  74.2, 110. ,  62. ,  65.2,  89. ,
        77. ,  87.7,  98. ,  84.9,  85. , 139.7, 120. , 125.7, 130. ,
       140. , 135. , 108. , 124. ,  95. ,  78. ,  86. ,  67.7, 102. ,
        92. ,  78.3,  80.7,  90.7,  77.7,  82.7,  62.7, 105. ,  63. ,
        66. ,  57. , 152. ,  79. ,  75.4,  73. ,  97.7,  58. ,  71.5,
       170. ,  84.5,  54.6,  57.7,  70.4,  64. ,  72.9,  56.5,  97.3,
        65.7,  65.4])

In [30]:
# Calculs départementaux
df = df.sort_values(by="dep")
df['NB'] = df.groupby('dep')['ps_id'].transform('nunique')
df["NB"].unique() # Jamais == nan

array([ 11,  26,   9,  10,   8, 113,   3,  19, 172,  29,   4,  36,   2,
        16,  25,  12,  21,  33,  39, 105,  96,  84,  46,  37,  57,  13,
        31,   5,  73,  28,  30,  14,   7,  50, 135,  55,  38,  61,  74,
        52, 100,  15,  23,  32, 173,  53,  58,  66,  18,  76,  89,  78,
        71], dtype=int64)

In [31]:
df['NB_F'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_F"].unique() # 3219

array([3219], dtype=int64)

In [32]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2'] = df.groupby("dep")["NB2"].transform("mean")
df["NB2"] = df["NB2"].fillna(0)
df["NB2"].unique()

array([  6.,   2.,   1.,   3.,  56.,   0.,  40.,   7.,   8.,   9.,   4.,
         5.,  16.,  48.,  10.,  12.,  20.,  24.,  47.,  30.,  17.,  72.,
       106.,  28.,  36.,  19.,  31.,  65.,  25.,  34.])

In [33]:
df['NB2_F'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_F'] = df['NB2_F'].mean()
df["NB2_F"].unique()  # 995

array([995.])

In [34]:
df['ShareS2'] = df['NB2'] / df['NB']
df['ShareS2_F'] = df['NB2_F'] / df['NB_F']
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102
157716,742917,H,COYAULT,JEAN MARC,CSC,3,3,1283,79.0,79.0,79.0,01283_0940_00019,2301,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.00,79.0,79.0,11,3219,6.0,995.0,0.545455,0.309102
64317,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2301,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102
190428,699249,F,EZ ZEMMAM,CAMELIA,CSC,3,1,1053,0.0,0.0,0.0,01053_1950_00060,2301,False,1000,1,699249_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102
190252,648335,H,TANAZACQ,REMI,CSC,3,1,1053,80.0,47.7,80.7,01053_1950_00060,2301,True,1000,1,648335_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,80.0,80.0,11,3219,6.0,995.0,0.545455,0.309102
82440,764848,F,BRUN,SOPHIE,CSC,3,3,1333,80.7,70.0,80.7,01333_0265_00060,2301,False,1390,1,764848_01333_0265_00060,1,1,23,10.0,1,4533,1517.0,1.00,80.7,80.7,11,3219,6.0,995.0,0.545455,0.309102
178301,788560,F,BOUREZG,ASMAA,CSC+MCC,3,3,1053,70.0,51.0,70.0,01053_1950_00060,2301,True,1000,1,788560_01053_1950_00060,1,1,23,10.0,1,4533,1517.0,1.00,70.0,70.0,11,3219,6.0,995.0,0.545455,0.309102
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7,100.7,11,3219,6.0,995.0,0.545455,0.309102
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102


In [35]:
# Dépassement
df['exessr'] = ((df['prixmoyen'] - tarif_s1) / tarif_s1) * 100
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000
157716,742917,H,COYAULT,JEAN MARC,CSC,3,3,1283,79.0,79.0,79.0,01283_0940_00019,2301,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.00,79.0,79.0,11,3219,6.0,995.0,0.545455,0.309102,54.901961
64317,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2301,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000
190428,699249,F,EZ ZEMMAM,CAMELIA,CSC,3,1,1053,0.0,0.0,0.0,01053_1950_00060,2301,False,1000,1,699249_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000
190252,648335,H,TANAZACQ,REMI,CSC,3,1,1053,80.0,47.7,80.7,01053_1950_00060,2301,True,1000,1,648335_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,80.0,80.0,11,3219,6.0,995.0,0.545455,0.309102,56.862745
82440,764848,F,BRUN,SOPHIE,CSC,3,3,1333,80.7,70.0,80.7,01333_0265_00060,2301,False,1390,1,764848_01333_0265_00060,1,1,23,10.0,1,4533,1517.0,1.00,80.7,80.7,11,3219,6.0,995.0,0.545455,0.309102,58.235294
178301,788560,F,BOUREZG,ASMAA,CSC+MCC,3,3,1053,70.0,51.0,70.0,01053_1950_00060,2301,True,1000,1,788560_01053_1950_00060,1,1,23,10.0,1,4533,1517.0,1.00,70.0,70.0,11,3219,6.0,995.0,0.545455,0.309102,37.254902
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7,100.7,11,3219,6.0,995.0,0.545455,0.309102,97.450980
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000


In [36]:
for seuil in [10, 25, 50, 75, 100]:
    df[f'c{seuil}'] = df.groupby('dep')['exessr'].transform(lambda x: (x >= seuil).sum())
    df[f'c{seuil}_F'] = (df['exessr'] >= seuil).sum()
    df[f'r{seuil}'] = (df[f'c{seuil}'] / df['NB']) * 100
    df[f'r{seuil}_F'] = df[f'c{seuil}_F'] / df['NB_F']
print("c10", df["c10"].unique())
print("c25", df["c25"].unique())
print("c50", df["c50"].unique())
print("c100", df["c100"].unique())
print("cX0_F", df["c10_F"].unique(), df["c25_F"].unique(), df["c50_F"].unique(), df["c100_F"].unique())
print("rX0_F", df["r10_F"].unique(), df["r25_F"].unique(), df["r50_F"].unique(), df["r100_F"].unique())
df.head(20)

c10 [ 5  1  0 32  7  2  3 19  4 12 14 56 11 84 34 16 58 21 10]
c25 [ 5  1  0 29  2  3 16 12  7  4 13 53 10 82 11 32 14 58 21  9]
c50 [ 4  0 13  1  3  2  5  6  7 28 78 26 11 46 14]
c100 [ 0  5  1 32 11  3]
cX0_F [482] [457] [287] [54]
rX0_F [0.14973594] [0.14196956] [0.08915812] [0.0167754]


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
157716,742917,H,COYAULT,JEAN MARC,CSC,3,3,1283,79.0,79.0,79.0,01283_0940_00019,2301,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.00,79.0,79.0,11,3219,6.0,995.0,0.545455,0.309102,54.901961,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
64317,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2301,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
190428,699249,F,EZ ZEMMAM,CAMELIA,CSC,3,1,1053,0.0,0.0,0.0,01053_1950_00060,2301,False,1000,1,699249_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
190252,648335,H,TANAZACQ,REMI,CSC,3,1,1053,80.0,47.7,80.7,01053_1950_00060,2301,True,1000,1,648335_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.50,80.0,80.0,11,3219,6.0,995.0,0.545455,0.309102,56.862745,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
82440,764848,F,BRUN,SOPHIE,CSC,3,3,1333,80.7,70.0,80.7,01333_0265_00060,2301,False,1390,1,764848_01333_0265_00060,1,1,23,10.0,1,4533,1517.0,1.00,80.7,80.7,11,3219,6.0,995.0,0.545455,0.309102,58.235294,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
178301,788560,F,BOUREZG,ASMAA,CSC+MCC,3,3,1053,70.0,51.0,70.0,01053_1950_00060,2301,True,1000,1,788560_01053_1950_00060,1,1,23,10.0,1,4533,1517.0,1.00,70.0,70.0,11,3219,6.0,995.0,0.545455,0.309102,37.254902,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
178322,648117,H,PIQUERAS,ERIC,CSC,4,3,1053,100.7,47.7,100.7,01053_1950_00060,2301,True,1000,1,648117_01053_1950_00060,3,3,23,10.0,1,4533,1517.0,0.33,100.7,100.7,11,3219,6.0,995.0,0.545455,0.309102,97.450980,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
266884,648110,H,ABDULMALAK,ABDUL NASSER,CSC+MCC,3,1,1033,0.0,0.0,0.0,01033_0227_00005,2301,False,1200,1,648110_01033_0227_00005,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775
95024,648079,H,BERRICHE,AMINE,CSC+MCC,2,1,1142,0.0,0.0,0.0,01142_0186_01100,2301,False,1120,1,648079_01142_0186_01100,1,1,23,10.0,1,4533,1517.0,1.00,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775


****dépassement moyen par département (uniquement les acteurs  faisant du dépassement)

In [37]:
# Moyennes nationales
df['PF'] = df['prixmoyen'].mean() # 55.86
df["PF"].unique()

array([55.86327581])

In [38]:
df['PFS2'] = df[mask_s2]['prixmoyen'].mean()
df['PFS2'].unique()  # 66.55

array([66.55024201])

In [39]:
# Moyennes départementales
df['PrixMoyen'] = df.groupby('dep')['prixmoyen'].transform('mean')
df['PrixMoyen'].unique()

array([65.12727273, 51.65384615, 51.        , 59.67433628, 54.22222222,
       52.01685393, 53.96896552, 51.36363636, 52.        , 51.65757576,
       51.25      , 56.26315789, 55.        , 52.29230769, 54.55048544,
       51.67058824, 51.62      , 52.16216216, 56.72105263, 52.26315789,
       51.5483871 , 52.98630137, 54.53448276, 60.        , 52.67142857,
       53.2875    , 58.24642857, 56.27702703, 51.36538462, 65.11121495,
       58.090625  , 75.78418079, 53.03148148, 56.2362069 , 67.89705882,
       51.77777778, 56.54383562, 51.48717949, 54.61904762, 58.33636364,
       53.97      , 56.00897436, 77.80659341, 53.56025641, 59.04861111,
       57.80212766])

In [40]:
df['PrixMoyenS2'] = df[mask_s2].groupby('dep')['prixmoyen'].transform('mean')
df["PrixMoyenS2"] = df.groupby('dep')['PrixMoyenS2'].transform('mean')
df['PrixMoyenS2'].unique()

array([76.9       , 59.5       , 51.        , 68.50357143, 60.66666667,
               nan, 55.20930233, 63.3       , 55.        , 55.22222222,
       52.97272727, 53.        , 71.        , 75.        , 67.8       ,
       57.77222222, 56.7       , 53.38461538, 56.375     , 67.305     ,
       55.8       , 53.125     , 57.04166667, 68.08333333, 66.75      ,
       62.7       , 57.61914894, 71.29      , 64.01666667, 52.11764706,
       70.11265823, 62.345     , 91.61851852, 57.85625   , 61.84642857,
       82.05405405, 72.3       , 53.11111111, 63.66666667, 67.14      ,
       80.7       , 63.209375  , 87.96060606, 58.988     , 68.04411765,
       77.64166667])

In [41]:
# df['Prix'] = df.groupby('dep')['PrixMoyen'].transform('mean')
df['depmoyen'] = ((df['PrixMoyen'] - tarif_s1) / tarif_s1) * 100  # * 100.roound(5)
df["depmoyen"].unique()

array([27.70053476,  1.28205128,  0.        , 17.00850252,  6.31808279,
        1.99383124,  5.82150101,  0.71301248,  1.96078431,  1.28936423,
        0.49019608, 10.31991744,  7.84313725,  2.53393665,  6.96173615,
        1.31487889,  1.21568627,  2.27874934, 11.21775026,  2.47678019,
        1.07526882,  3.89470857,  6.93035835, 17.64705882,  3.27731092,
        4.48529412, 14.20868347, 10.34711182,  0.71644042, 27.66904893,
       13.90318627, 48.59643292,  3.98329702, 10.26707235, 33.13148789,
        1.52505447, 10.87026591,  0.9552539 ,  7.0961718 , 14.38502674,
        5.82352941,  9.82151835, 52.56194786,  5.02011061, 15.78159041,
       13.33750521])

In [42]:
#9.53
df["depmoyen_F"] =((df['PF'] - tarif_s1) / tarif_s1) * 100
df["depmoyen_F"] = df["depmoyen_F"].round(3)
df["depmoyen_F"].unique()

array([9.536])

In [43]:
df['depmoyenS2'] = ((df['PrixMoyenS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyenS2'] = df['depmoyenS2'].round(3)
df['depmoyenS2'].unique()

array([50.784, 16.667,  0.   , 34.321, 18.954,    nan,  8.254, 24.118,
        7.843,  8.279,  3.868,  3.922, 39.216, 47.059, 32.941, 13.279,
       11.176,  4.676, 10.539, 31.971,  9.412,  4.167, 11.846, 33.497,
       30.882, 22.941, 12.979, 39.784, 25.523,  2.191, 37.476, 22.245,
       79.644, 13.444, 21.268, 60.89 , 41.765,  4.139, 24.837, 31.647,
       58.235, 23.94 , 72.472, 15.663, 33.42 , 52.239])

In [44]:
df['depmoyen_FS2'] = ((df['PFS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyen_FS2'].unique() # 30.49


array([30.49067061])

In [45]:
df = df.sort_values(by=['dep', 'depmoyen'], ascending=[True, False])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
289612,647961,H,FAVIA,AGOSTINO,CSC,3,1,1354,0.0,0.0,0.0,01354_0245_00110,2301,False,1630,1,647961_01354_0245_00110,1,1,23,10.0,1,4533,1517.0,1.0,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775,55.863276,66.550242,65.127273,76.9,27.700535,9.536,50.784,30.490671
157716,742917,H,COYAULT,JEAN MARC,CSC,3,3,1283,79.0,79.0,79.0,01283_0940_00019,2301,False,1100,1,742917_01283_0940_00019,1,1,23,10.0,1,4533,1517.0,1.0,79.0,79.0,11,3219,6.0,995.0,0.545455,0.309102,54.901961,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775,55.863276,66.550242,65.127273,76.9,27.700535,9.536,50.784,30.490671
64317,757551,H,HALARY,JEAN JACQUES,CSC,4,1,1283,0.0,0.0,0.0,01283_0800_00029,2301,False,1100,1,757551_01283_0800_00029,1,1,23,10.0,1,4533,1517.0,1.0,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775,55.863276,66.550242,65.127273,76.9,27.700535,9.536,50.784,30.490671
190428,699249,F,EZ ZEMMAM,CAMELIA,CSC,3,1,1053,0.0,0.0,0.0,01053_1950_00060,2301,False,1000,1,699249_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.5,51.0,51.0,11,3219,6.0,995.0,0.545455,0.309102,0.000000,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775,55.863276,66.550242,65.127273,76.9,27.700535,9.536,50.784,30.490671
190252,648335,H,TANAZACQ,REMI,CSC,3,1,1053,80.0,47.7,80.7,01053_1950_00060,2301,True,1000,1,648335_01053_1950_00060,2,2,23,10.0,1,4533,1517.0,0.5,80.0,80.0,11,3219,6.0,995.0,0.545455,0.309102,56.862745,5,482,45.454545,0.149736,5,457,45.454545,0.14197,4,287,36.363636,0.089158,1,161,9.090909,0.050016,0,54,0.0,0.016775,55.863276,66.550242,65.127273,76.9,27.700535,9.536,50.784,30.490671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92550,724709,F,THO AGOSTINI,AURELIA,CSC,4,1,2B033,0.0,0.0,0.0,2b033_0400_00006,2301,False,20200,202,724709_2b033_0400_00006,1,1,12,1.0,1,4533,1517.0,1.0,51.0,51.0,10,3219,1.0,995.0,0.100000,0.309102,0.000000,0,482,0.000000,0.149736,0,457,0.000000,0.14197,0,287,0.000000,0.089158,0,161,0.000000,0.050016,0,54,0.0,0.016775,55.863276,66.550242,51.000000,51.0,0.000000,9.536,0.000,30.490671
92438,659295,H,RICCI,DOMINIQUE,CSC,3,1,2B033,0.0,0.0,0.0,2b033_0400_00006,2301,False,20200,202,659295_2b033_0400_00006,1,1,12,1.0,1,4533,1517.0,1.0,51.0,51.0,10,3219,1.0,995.0,0.100000,0.309102,0.000000,0,482,0.000000,0.149736,0,457,0.000000,0.14197,0,287,0.000000,0.089158,0,161,0.000000,0.050016,0,54,0.0,0.016775,55.863276,66.550242,51.000000,51.0,0.000000,9.536,0.000,30.490671
32924,659480,H,VELUT,JEAN GABRIEL,CSC+MCC,3,1,2B134,0.0,0.0,0.0,2b134_a087_00001,2301,False,20220,202,659480_2b134_a087_00001,1,1,12,1.0,1,4533,1517.0,1.0,51.0,51.0,10,3219,1.0,995.0,0.100000,0.309102,0.000000,0,482,0.000000,0.149736,0,457,0.000000,0.14197,0,287,0.000000,0.089158,0,161,0.000000,0.050016,0,54,0.0,0.016775,55.863276,66.550242,51.000000,51.0,0.000000,9.536,0.000,30.490671
92470,768993,H,RAFFALLI,GILLES,CSC,3,1,2B033,0.0,0.0,0.0,2b033_0400_00006,2301,False,20200,202,768993_2b033_0400_00006,1,1,12,1.0,1,4533,

In [46]:
df = df.sort_values(by="dep", ascending=False)
df = df.drop_duplicates(subset=['dep'])
df['depmoyen'] = df['depmoyen'].fillna(0)
pd.set_option('display.max_rows', 20)
df = df.sort_values(by="dep")
df['depmoyen'].unique()
#df.head(20)

array([27.70053476,  1.28205128,  0.        , 17.00850252,  6.31808279,
        1.99383124,  5.82150101,  0.71301248,  1.96078431,  1.28936423,
        0.49019608, 10.31991744,  7.84313725,  2.53393665,  6.96173615,
        1.31487889,  1.21568627,  2.27874934, 11.21775026,  2.47678019,
        1.07526882,  3.89470857,  6.93035835, 17.64705882,  3.27731092,
        4.48529412, 14.20868347, 10.34711182,  0.71644042, 27.66904893,
       13.90318627, 48.59643292,  3.98329702, 10.26707235, 33.13148789,
        1.52505447, 10.87026591,  0.9552539 ,  7.0961718 , 14.38502674,
        5.82352941,  9.82151835, 52.56194786,  5.02011061, 15.78159041,
       13.33750521])

In [47]:
# gender nom prénom naturedexercice convention optioncontratdaccèsauxsoins codeccamdelacte ps_id montantgénéralementconstaté borneinférieuredumontant bornesupérieuredumontant date_source_id adresse_id matchcp codeinsee b unique un weight mp prixmoyen exessr nb10 c10 nb25 c25 nb50 c50 nb75 c75 nb100 c100 exess
df = df.drop(["gender","nom","prénom","naturedexercice","convention","optioncontratdaccèsauxsoins","codeccamdelacte","ps_id","montantgénéralementconstaté","borneinférieuredumontant","bornesupérieuredumontant","date_source_id","adresse_id","matchcp","codeinsee","b","unique","un","weight","mp","prixmoyen","exessr","c10","c25","c50","c75","c100"], axis=1)
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
178301,1,23,10.0,1,4533,1517.0,11,3219,6.0,995.0,0.545455,0.309102,482,45.454545,0.149736,457,45.454545,0.14197,287,36.363636,0.089158,161,9.090909,0.050016,54,0.000000,0.016775,55.863276,66.550242,65.127273,76.900000,27.700535,9.536,50.784,30.490671
202441,2,29,2.0,1,4533,1517.0,26,3219,2.0,995.0,0.076923,0.309102,482,3.846154,0.149736,457,3.846154,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.653846,59.500000,1.282051,9.536,16.667,30.490671
157400,3,21,2.0,1,4533,1517.0,9,3219,2.0,995.0,0.222222,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671
247524,4,13,2.0,1,4533,1517.0,10,3219,1.0,995.0,0.100000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671
304712,5,10,4.0,1,4533,1517.0,8,3219,3.0,995.0,0.375000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197316,93,126,53.0,1,4533,1517.0,78,3219,25.0,995.0,0.320513,0.309102,482,8.974359,0.149736,457,8.974359,0.14197,287,6.410256,0.089158,161,1.282051,0.050016,54,0.000000,0.016775,55.863276,66.550242,53.560256,58.988000,5.020111,9.536,15.663,30.490671
265789,94,95,48.0,1,4533,1517.0,71,3219,34.0,995.0,0.478873,0.309102,482,29.577465,0.149736,457,29.577465,0.14197,287,19.718310,0.089158,161,5.633803,0.050016,54,0.000000,0.016775,55.863276,66.550242,59.048611,68.044118,15.781590,9.536,33.420,30.490671
45841,95,57,15.0,1,4533,1517.0,46,3219,12.0,995.0,0.260870,0.309102,482,21.739130,0.149736,457,19.565217,0.14197,287,8.695652,0.089158,161,8.695652,0.050016,54,6.521739,0.016775,55.863276,66.550242,57.802128,77.641667,13.337505,9.536,52.239,30.490671
37542,201,13,8.0,1,4533,1517.0,3,3219,0.0,995.0,0.000000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,NaN,0.000000,9.536,NaN,30.490671


In [48]:
# Séparer les lignes à dupliquer
df_to_duplicate = df[df['dep'] == 75].copy()
df_to_duplicate['dup'] = 1  # marquer les duplications
# Marquer les lignes originales
df['dup'] = 0
# Fusionner les deux
df = pd.concat([df, df_to_duplicate], ignore_index=True)
df



,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,23,10.0,1,4533,1517.0,11,3219,6.0,995.0,0.545455,0.309102,482,45.454545,0.149736,457,45.454545,0.14197,287,36.363636,0.089158,161,9.090909,0.050016,54,0.000000,0.016775,55.863276,66.550242,65.127273,76.900000,27.700535,9.536,50.784,30.490671,0
1,2,29,2.0,1,4533,1517.0,26,3219,2.0,995.0,0.076923,0.309102,482,3.846154,0.149736,457,3.846154,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.653846,59.500000,1.282051,9.536,16.667,30.490671,0
2,3,21,2.0,1,4533,1517.0,9,3219,2.0,995.0,0.222222,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671,0
3,4,13,2.0,1,4533,1517.0,10,3219,1.0,995.0,0.100000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671,0
4,5,10,4.0,1,4533,1517.0,8,3219,3.0,995.0,0.375000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,95,48.0,1,4533,1517.0,71,3219,34.0,995.0,0.478873,0.309102,482,29.577465,0.149736,457,29.577465,0.14197,287,19.718310,0.089158,161,5.633803,0.050016,54,0.000000,0.016775,55.863276,66.550242,59.048611,68.044118,15.781590,9.536,33.420,30.490671,0
93,95,57,15.0,1,4533,1517.0,46,3219,12.0,995.0,0.260870,0.309102,482,21.739130,0.149736,457,19.565217,0.14197,287,8.695652,0.089158,161,8.695652,0.050016,54,6.521739,0.016775,55.863276,66.550242,57.802128,77.641667,13.337505,9.536,52.239,30.490671,0
94,201,13,8.0,1,4533,1517.0,3,3219,0.0,995.0,0.000000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,NaN,0.000000,9.536,NaN,30.490671,0
95,202,12,1.0,1,4533,1517.0,10,3219,1.0,995.0,0.100000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000,30.490671,0


In [49]:
df.loc[df["dup"] == 1, "dep"] = 0
df.loc[df["dup"] == 1, "NB_total"] = df.loc[df["dup"] == 1]["NB_Ftotal"]
df.loc[df["dup"] == 1, "NB2_total"] = df.loc[df["dup"] == 1]["NB2_Ftotal"]
df.loc[df["dup"] == 1, "NB"] = df.loc[df["dup"] == 1]["NB_F"]
df.loc[df["dup"] == 1, "NB2"] = df.loc[df["dup"] == 1]["NB2_F"]
df.loc[df["dup"] == 1, "ShareS2"] = df.loc[df["dup"] == 1]["ShareS2_F"]
df.loc[df["dup"] == 1, "r10"] = df.loc[df["dup"] == 1]["r10_F"]
df.loc[df["dup"] == 1, "r25"] = df.loc[df["dup"] == 1]["r25_F"]
df.loc[df["dup"] == 1, "r50"] = df.loc[df["dup"] == 1]["r50_F"]
df.loc[df["dup"] == 1, "r100"] = df.loc[df["dup"] == 1]["r100_F"]
df.loc[df["dup"] == 1, "PrixMoyen"] = df.loc[df["dup"] == 1]["PF"]
df.loc[df["dup"] == 1, "PrixMoyenS2"] = df.loc[df["dup"] == 1]["PFS2"]
df.loc[df["dup"] == 1, "depmoyen"] = df.loc[df["dup"] == 1]["depmoyen_F"]
df.loc[df["dup"] == 1, "depmoyenS2"] = df.loc[df["dup"] == 1]["depmoyen_FS2"]
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,23,10.0,1,4533,1517.0,11,3219,6.0,995.0,0.545455,0.309102,482,45.454545,0.149736,457,45.454545,0.14197,287,36.363636,0.089158,161,9.090909,0.050016,54,0.000000,0.016775,55.863276,66.550242,65.127273,76.900000,27.700535,9.536,50.784000,30.490671,0
1,2,29,2.0,1,4533,1517.0,26,3219,2.0,995.0,0.076923,0.309102,482,3.846154,0.149736,457,3.846154,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.653846,59.500000,1.282051,9.536,16.667000,30.490671,0
2,3,21,2.0,1,4533,1517.0,9,3219,2.0,995.0,0.222222,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000000,30.490671,0
3,4,13,2.0,1,4533,1517.0,10,3219,1.0,995.0,0.100000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000000,30.490671,0
4,5,10,4.0,1,4533,1517.0,8,3219,3.0,995.0,0.375000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000000,30.490671,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,95,48.0,1,4533,1517.0,71,3219,34.0,995.0,0.478873,0.309102,482,29.577465,0.149736,457,29.577465,0.14197,287,19.718310,0.089158,161,5.633803,0.050016,54,0.000000,0.016775,55.863276,66.550242,59.048611,68.044118,15.781590,9.536,33.420000,30.490671,0
93,95,57,15.0,1,4533,1517.0,46,3219,12.0,995.0,0.260870,0.309102,482,21.739130,0.149736,457,19.565217,0.14197,287,8.695652,0.089158,161,8.695652,0.050016,54,6.521739,0.016775,55.863276,66.550242,57.802128,77.641667,13.337505,9.536,52.239000,30.490671,0
94,201,13,8.0,1,4533,1517.0,3,3219,0.0,995.0,0.000000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,NaN,0.000000,9.536,NaN,30.490671,0
95,202,12,1.0,1,4533,1517.0,10,3219,1.0,995.0,0.100000,0.309102,482,0.000000,0.149736,457,0.000000,0.14197,287,0.000000,0.089158,161,0.000000,0.050016,54,0.000000,0.016775,55.863276,66.550242,51.000000,51.000000,0.000000,9.536,0.000000,30.490671,0


In [50]:
# NB_Ftotal  NB2_Ftotal NB_F NB2_F ShareS2_F r10_F r25_F r50_F r100_F PF PFS2 depmoyen_F depmoyen_FS2 c10_F c25_F c50_F c100_F dup c
df = df.drop(["NB_Ftotal","NB2_Ftotal","NB_F","NB2_F","ShareS2_F","r10_F","r25_F","r50_F","r100_F","PF","PFS2","depmoyen_F","depmoyen_FS2","c10_F","c25_F","c50_F","c75_F","r75_F", "r75","c100_F","dup","c"], axis=1)
df

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,23,10.0,11,6.0,0.545455,45.454545,45.454545,36.363636,0.000000,65.127273,76.900000,27.700535,50.784000
1,2,29,2.0,26,2.0,0.076923,3.846154,3.846154,0.000000,0.000000,51.653846,59.500000,1.282051,16.667000
2,3,21,2.0,9,2.0,0.222222,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000000
3,4,13,2.0,10,1.0,0.100000,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000000
4,5,10,4.0,8,3.0,0.375000,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,95,48.0,71,34.0,0.478873,29.577465,29.577465,19.718310,0.000000,59.048611,68.044118,15.781590,33.420000
93,95,57,15.0,46,12.0,0.260870,21.739130,19.565217,8.695652,6.521739,57.802128,77.641667,13.337505,52.239000
94,201,13,8.0,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,NaN,0.000000,NaN
95,202,12,1.0,10,1.0,0.100000,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000000


In [51]:
# df[df["dep"] == 38]
df = df.reset_index(drop=True)
df.head(10)


,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,23,10.0,11,6.0,0.545455,45.454545,45.454545,36.363636,0.000000,65.127273,76.900000,27.700535,50.784
1,2,29,2.0,26,2.0,0.076923,3.846154,3.846154,0.000000,0.000000,51.653846,59.500000,1.282051,16.667
2,3,21,2.0,9,2.0,0.222222,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000
3,4,13,2.0,10,1.0,0.100000,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000
4,5,10,4.0,8,3.0,0.375000,0.000000,0.000000,0.000000,0.000000,51.000000,51.000000,0.000000,0.000
5,6,140,72.0,113,56.0,0.495575,28.318584,25.663717,11.504425,4.424779,59.674336,68.503571,17.008503,34.321
6,7,16,5.0,9,3.0,0.333333,11.111111,11.111111,11.111111,0.000000,54.222222,60.666667,6.318083,18.954
7,8,11,0.0,10,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,NaN,0.000000,NaN
8,9,3,0.0,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,NaN,0.000000,NaN
9,10,13,0.0,11,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,NaN,0.000000,NaN


In [52]:
df2 = pd.read_csv("../data/depassement/Rendus_PartIII/dépassement_cardiologue.csv")
df2.head(10)

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,depmoyen,PrixMoyenS2,depmoyenS2
0,1,23,10,11,6,0.545455,45.454544,45.454544,36.363636,0.000000,65.127274,27.701,76.900002,50.784
1,2,29,2,26,2,0.076923,3.846154,3.846154,0.000000,0.000000,51.653847,1.282,59.500000,16.667
2,3,21,2,9,2,0.222222,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,51.000000,0.000
3,4,13,2,10,1,0.100000,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,51.000000,0.000
4,5,10,4,8,3,0.375000,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,51.000000,0.000
5,6,140,72,113,56,0.495575,28.318584,25.663717,11.504425,4.424779,59.674335,17.009,68.503571,34.321
6,7,16,5,9,3,0.333333,11.111111,11.111111,11.111111,0.000000,54.222221,6.318,60.666668,18.954
7,8,11,0,10,0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,NaN,NaN
8,9,3,0,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,NaN,NaN
9,10,13,0,11,0,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,0.000,NaN,NaN


In [53]:
df1 = df.fillna(9999).round(2)
df2 = df2.fillna(9999).round(2)
diff = df1 - df2
pd.set_option('display.max_rows', 100)
diff.head(96)


,NB,NB2,NB2_total,NB_total,PrixMoyen,PrixMoyenS2,ShareS2,dep,depmoyen,depmoyenS2,r10,r100,r25,r50
0,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
5,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
6,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
7,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
8,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
9,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0


In [54]:
diff = df1["NB"].equals(df2["NB"])
diff = df1[df1["NB2"] != df2["NB2"]]
diff = df1[df1["NB2_total"] != df2["NB2_total"]]
diff = df1[df1["NB_total"] != df2["NB_total"]]
diff = df1[df1["PrixMoyen"] != df2["PrixMoyen"]]
diff = df1[df1["PrixMoyenS2"] != df2["PrixMoyenS2"]] # Aucune différence avec STATA, petite différence avec le fichier, + grosse différence avec SQL
diff

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
36,38,83,28.0,57,20.0,0.35,21.05,21.05,8.77,0.0,56.72,67.3,11.22,31.97


In [55]:
df.to_csv(f"../data/depassement/out/depassement_{profession_type}_{datesource_min}_{datesource_max}_{tarif_s1}_{study_id}.csv", index=False)

In [56]:
conn = psycopg2.connect(config.connection_string)
sql = f"delete from depassement where depassement_study_id={study_id}" # Ne marche pas la 1ère fois
with conn.cursor() as cur:
    cur.execute(sql)
conn.commit()
conn.close()
df["depassement_study_id"] = study_id
df.to_sql("depassement", config.connection_string, if_exists="append", index_label="id")


97